In [1]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')
infinity = solver.infinity()

# Variables
for i in range(1, 25, 1):
  exec(f'x{i} = solver.IntVar(0.0, infinity, "x{i}")', globals())
  exec(f'y{i} = solver.IntVar(0.0, infinity, "y{i}")', globals())
  exec(f'z{i} = solver.IntVar(0.0, infinity, "z{i}")', globals())

X = solver.IntVar(0.0, infinity, "X")
Y = solver.IntVar(0.0, infinity, "Y")
Z = solver.IntVar(0.0, infinity, "Z")

print('Number of variables =', solver.NumVariables())
assert solver.NumVariables() == 75

Number of variables = 75


In [2]:
# Constraints

# 0*x1 + 1*x2 + 1*x3 + 1*x4 ... -21X <=0
# 0*y1 + 0*y2 + 0*y3 + 0*y4 + 0*y5... -9Y <=0
# 0*z1 + 0*z2 + 0*z3 + 0*z4 + 0*z5 + 0*z6 + 1*z7... -8Z <=0
# Cycle above 24 times, each time move the 0s forward by 1, x: 1 0, y: 14 0, z: 15 0

x_no_0 = 1
y_no_0 = 14
z_no_0 = 15

for i in range(24):
  # Make cyclic zero masks
  multiply_x = [1] * 24
  multiply_y = [1] * 24
  multiply_z = [1] * 24
  if i + x_no_0 <= 24:
    multiply_x[i:i + x_no_0] = [0] * x_no_0
  else:  # fill rest of list with 0 first then cycle back to front
    multiply_x[i:] = [0] * (24 - i)
    multiply_x[:i + x_no_0 - 24] = [0] * (i + x_no_0 - 24)

  if i + y_no_0 <= 24:
    multiply_y[i:i + y_no_0] = [0] * y_no_0
  else:
    multiply_y[i:] = [0] * (24 - i)
    multiply_y[:i + y_no_0 - 24] = [0] * (i + y_no_0 - 24)

  if i + z_no_0 <= 24:
    multiply_z[i:i + z_no_0] = [0] * z_no_0
  else:
    multiply_z[i:] = [0] * (24 - i)
    multiply_z[:i + z_no_0 - 24] = [0] * (i + z_no_0 - 24)

  # Add the constraints
  # Total constraints for this: 24*3 = 72
  x_sum_expr_code = [f'x{j+1} * multiply_x[{j}]' for j in range(24)]
  exec(f'solver.Add(solver.Sum([{",".join(x_sum_expr_code)}]) <= 21*X)', globals())
  y_sum_expr_code = [f'y{j+1} * multiply_y[{j}]' for j in range(24)]
  exec(f'solver.Add(solver.Sum([{",".join(y_sum_expr_code)}]) <= 9*Y)', globals())
  z_sum_expr_code = [f'z{j+1} * multiply_z[{j}]' for j in range(24)]
  exec(f'solver.Add(solver.Sum([{",".join(z_sum_expr_code)}]) <= 8*Z)', globals())

# xi <= X for all x
# yi <= Y for all y
# zi <= Z for all z
# Total constraints for this: 24*3 = 72
for i in range(1, 25, 1):
  exec(f'solver.Add(x{i} <= X)', globals())
  exec(f'solver.Add(y{i} <= Y)', globals())
  exec(f'solver.Add(z{i} <= Z)', globals())

# x1+y1+z1 = K, for var1...24
# Total constraints for this: 24
K = [172,205,201,190,170,205,173,179,176,200,199,199,190,208,194,184,208,209,195,193,197,207,209,188]  # Obtained from Excel provided
i = 0
for k in K:
  i += 1
  exec(f'solver.Add(solver.Sum([x{i}, y{i}, z{i}]) == {k})', globals())

print('Number of constraints =', solver.NumConstraints())
assert solver.NumConstraints() == 72 * 2 + 24

Number of constraints = 168


In [3]:
solver.Minimize(10400 * X + 9000 * Y + 8200 * Z)
status = solver.Solve()

In [4]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', round(solver.Objective().Value()))
    for i in range(1, 25, 1):
        exec(f'print("x{i} =", int(x{i}.solution_value()))', globals())
        exec(f'print("y{i} =", int(y{i}.solution_value()))', globals())
        exec(f'print("z{i} =", int(z{i}.solution_value()))', globals())
    print('X =', int(X.solution_value()))
    print('Y =', int(Y.solution_value()))
    print('Z =', int(Z.solution_value()))
    print(f'Problem solved in {solver.wall_time()} milliseconds')
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 1842400
x1 = 0
y1 = 7
z1 = 165
x2 = 0
y2 = 7
z2 = 198
x3 = 0
y3 = 7
z3 = 194
x4 = 0
y4 = 0
z4 = 190
x5 = 0
y5 = 0
z5 = 170
x6 = 0
y6 = 7
z6 = 198
x7 = 0
y7 = 0
z7 = 173
x8 = 0
y8 = 0
z8 = 179
x9 = 0
y9 = 0
z9 = 176
x10 = 0
y10 = 7
z10 = 193
x11 = 0
y11 = 7
z11 = 192
x12 = 0
y12 = 7
z12 = 192
x13 = 0
y13 = 3
z13 = 187
x14 = 0
y14 = 3
z14 = 205
x15 = 0
y15 = 7
z15 = 187
x16 = 0
y16 = 7
z16 = 177
x17 = 0
y17 = 7
z17 = 201
x18 = 0
y18 = 7
z18 = 202
x19 = 0
y19 = 7
z19 = 188
x20 = 0
y20 = 7
z20 = 186
x21 = 0
y21 = 7
z21 = 190
x22 = 0
y22 = 7
z22 = 200
x23 = 0
y23 = 4
z23 = 205
x24 = 0
y24 = 1
z24 = 187
X = 0
Y = 7
Z = 217
Problem solved in 60 milliseconds
